In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['basePart'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=' + _ for _ in list_url]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                resp = requests.get(url, proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                info = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                part_number = info['pd']['basePart']
                
                # = = = = = = = = = = = = = = =
                
                part_type = info['pd']['partDesc_en']
                
                # = = = = = = = = = = = = = = =
                
                part_type_long = info['pd']['partLongDesc_en']
                
                # = = = = = = = = = = = = = = =
                
                brand = info['pd']['brand']
                
                # = = = = = = = = = = = = = = =
                
                pop = info['pd']['pop']
                
                # = = = = = = = = = = = = = = =
                
                per_car = info['pd']['qtyEach']
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                for _ in info['pp']['buyersGuides']:
                    list_vehicle.append(_['buyersGuideDesc'])

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Part_Type': part_type,
                                         'Part_Type_Long': part_type_long,
                                         'Brand': brand,
                                         'Pop': pop,
                                         'Per_Car': per_car,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': ''}])
                
                status = 'ok'
                
                # = = = = = = = = = = = = = = =
                
                for name_value in info['pp']['partSpecs']:
                    df_temp[name_value['attributeName_en'].lower()] = name_value['siteValue_en']
                
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：1

https://ecatalog.smpcorp.com/V2/STD/api/part/partselect?func=PART&vid=&part=ADR4368  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束


D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
